In [1]:
import pandas as pd
import json
from mplsoccer import Pitch

DATA_FOLDER = "data"

### Get the matches

In [16]:
# Get all matches
matches_df = pd.read_parquet(f"{DATA_FOLDER}/matches.parquet")

# Create new column with match name (team names)
matches_df["match"] = (
    matches_df["home_team"].apply(lambda x: x["short_name"])
    + " vs "
    + matches_df["away_team"].apply(lambda x: x["short_name"])
)

# Create data column with match date
matches_df["date"] = pd.to_datetime(matches_df["date_time"]).dt.date

matches_df.head()

,id,date_time,home_team,away_team,status,competition_id,season_id,competition_edition_id,match,date
0,2014987,2025-05-04T12:00:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 285, 'short_name': 'Celta Vigo'}",closed,4,95,900,Real Madrid vs Celta Vigo,2025-05-04
1,2012892,2025-04-23T19:30:00Z,"{'id': 268, 'short_name': 'Getafe CF'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,95,900,Getafe CF vs Real Madrid,2025-04-23
2,2012431,2025-04-20T19:00:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 273, 'short_name': 'Athletic Bilbao'}",closed,4,95,900,Real Madrid vs Athletic Bilbao,2025-04-20
3,2011844,2025-04-16T19:00:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 3, 'short_name': 'Arsenal'}",closed,10,95,948,Real Madrid vs Arsenal,2025-04-16
4,2011428,2025-04-13T14:15:00Z,"{'id': 313, 'short_name': 'Alavés'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,95,900,Alavés vs Real Madrid,2025-04-13


### Get dynamic data for a certain match
For example: the last game in the dataset:
- id: 2014987
- match: Real Madrid vs Celta Vigo
- date: 2025-05-04

For a more detailed explanation: check the [Dynamic Events CSV Specifications file](./20250314%20-%20Dynamic%20Events%20CSV%20Specifications.pdf)

In [20]:
match_id = 2014987

# Get dynamic data for this match
dynamic_data = pd.read_parquet(f"{DATA_FOLDER}/dynamic/{match_id}.parquet")
dynamic_data.head(20)

,event_id,index,match_id,frame_start,frame_end,frame_physical_start,time_start,time_end,minute_start,second_start,...,xloss_player_possession_end,xloss_player_possession_max,xshot_player_possession_start,xshot_player_possession_end,xshot_player_possession_max,is_player_possession_start_matched,is_player_possession_end_matched,is_previous_pass_matched,is_pass_reception_matched,fully_extrapolated
0,8_0,0,2014987,23,23,NaN,00:01.3,00:01.3,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,True,True
1,8_1,1,2014987,40,52,NaN,00:03.0,00:04.2,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False
2,7_0,2,2014987,40,48,NaN,00:03.0,00:03.8,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False
3,7_1,3,2014987,49,52,NaN,00:03.9,00:04.2,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False
4,7_2,4,2014987,50,52,NaN,00:04.0,00:04.2,0,4,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False
5,7_3,5,2014987,50,52,NaN,00:04.0,00:04.2,0,4,...,NaN,NaN,NaN,NaN,NaN,True,True,None,True,False
6,7_4,6,2014987,51,52,NaN,00:04.1,00:04.2,0,4,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False
7,9_0,7,2014987,82,102,82.0,00:07.2,00:09.2,0,7,...,0.164,0.386,0.0,0.0,0.000,True,True,None,True,None
8,9_1,8,2014987,83,102,83.0,00:07.3,00:09.2,0,7,...,0.164,0.386,0.0,0.0,0.000,True,True,None,True,None
9,8_2,9,2014987,84,102,NaN,00:07.4,00:09.2,0,7,...,NaN,NaN,NaN,NaN,NaN,True,True,None,True,False
